Choose GPU

In [1]:
import os
KAGGLE = True
if os.name == "nt":
    KAGGLE = False
print(f"KAGGLE: {KAGGLE}")
if not KAGGLE:
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"

KAGGLE: False


In [2]:
# general
import gc
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

from glob import glob
import random
import numpy as np
import pandas as pd
import json
from IPython.core.display import display, HTML
if KAGGLE:
    from kaggle_datasets import KaggleDatasets

# ML
from sklearn.metrics import roc_auc_score

# DL
import tensorflow as tf
# tf.config.optimizer.set_jit(True)

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns



### Settings

In [3]:
# tf.compat.v1.disable_eager_execution()
display(HTML("<style>.container { width:90% !important; }</style>"))
%matplotlib inline
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

In [4]:
def auto_select_accelerator():
    TPU_DETECTED = False
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        tf.keras.mixed_precision.set_global_policy("mixed_bfloat16")
        print(f"Running on TPU: {tpu.master()}")
        TPU_DETECTED = True
    except ValueError:
        strategy = tf.distribute.get_strategy()
        tf.keras.mixed_precision.set_global_policy("mixed_float16")
    num_replicas = strategy.num_replicas_in_sync
    print(f"Running on {num_replicas} replica{'s' if num_replicas > 1 else ''}")
    return strategy, TPU_DETECTED, num_replicas

## PARAMS

In [5]:
strategy, TPU_Detected, REPLICAS = auto_select_accelerator()
INPUT_DIR = "../input/g2net-gravitational-wave-detection"
MDLS_PATH = "." if KAGGLE else "../models"
# TRAIN_FILES_PATH = "../input/filtered*_tfrec"
AUTO = tf.data.experimental.AUTOTUNE
tfrec_folders = ["filtered-whitened-tfrec"]#["filtered-tfrec", "filtered-whitened-tfrec", "filtered-whitened-inverted-tfrec", "whitened-tfrec"]


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6
Running on 1 replica


In [6]:
results_df = "no file"
if not KAGGLE:
    results_df = pd.read_csv("../models/results.csv", index_col=[0]).tail(20)
results_df

,lr,version,train_mode,batch_size,epochs,bavg_epoch,bavg_loss,bavg_auc,changelog,seed
23,0.00010,75,full,256,10,9,0.454001,0.830381,"doubled pool size, except for last, doubled nu...",42.0
24,0.00100,76,full,256,10,6,0.469206,0.832270,base model with same padding,42.0
25,0.00010,78,full,256,40,12,0.442443,0.841306,doubled last layer in all blocks. lr/10,42.0
26,0.00050,79,full,256,40,25,0.439258,0.842578,doubled last layer in all blocks doubled size ...,42.0
27,0.00050,83,full,256,40,27,0.439051,0.841536,added dropout layers to dense layers,42.0
28,0.00010,88,full,256,40,8,0.453092,0.834111,everything had 3 layers now and relu activation,42.0
29,0.00010,92,full,256,100,17,0.434836,0.832301,dropout layers,42.0
30,0.00010,110,full,256,100,11,0.470093,0.815836,dropout layers,42.0
31,0.00010,113,full,256,100,8,0.476270,0.813889,dropout layers,42.0
32,0.00002,119,full,256,100,36,0.463551,0.812370,new dataset using best model (base model with ...,42.0


In [7]:
if KAGGLE:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    user_credential = user_secrets.get_gcloud_credential()
    user_secrets.set_tensorflow_credential(user_credential)

In [8]:
if not os.path.exists(f"{MDLS_PATH}/results.csv"):
    VER = 1
else:
    results = pd.read_csv(f"{MDLS_PATH}/results.csv", index_col=[0])
    VER = int(results.version.max())
Params ={
    "lr": 0.0001 * REPLICAS,
    "version": VER,
    "train_mode": "full", #test, full
    "batch_size": 256 * REPLICAS,
    "epochs":100,
    "seed": 42,
    "changelog": "1/4 lr",
}
seed_everything(Params["seed"])

Make Model directory

In [9]:
VER = Params["version"]
MDL_PATH = f"{MDLS_PATH}/models_v{VER:03}"
while os.path.exists(MDL_PATH):
    VER += 1
    MDL_PATH = f"{MDLS_PATH}/models_v{VER:03}"
Params["version"]=VER
os.mkdir(MDL_PATH)

In [10]:
def random_cut(x, y):
    a = np.zeros([4096,3], dtype=np.float32)
    t0 = np.random.randint(0,512)
    a[t0:t0+(3584)] = x[t0:t0+(3584)]
    return a,y

In [11]:
import tensorflow_datasets as tfds
input_context = tf.distribute.InputContext(
    input_pipeline_id=1,
    num_input_pipelines = 8
)
read_config= tfds.ReadConfig(
    input_context=input_context
)

def load_dataset(files, shuffle=True, ordered=False, labeled = True, repeat=True, return_labels = False, cut = False):
    if ordered:
        dataset = tf.data.TFRecordDataset(files, num_parallel_reads=None)
    else:
        dataset = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)


    def _parse_function(example_proto):
        if labeled:
            keys_to_feature = {
                "TimeSeries":tf.io.FixedLenFeature([4096,3],tf.float32),
                "Target":tf.io.FixedLenFeature([], tf.int64, default_value=0)}
            if return_labels:
                keys_to_feature["id"]=tf.io.FixedLenFeature([],tf.string, default_value="")
        else:
            keys_to_feature = {
                "TimeSeries": tf.io.FixedLenFeature([4096,3],tf.float32)
            }
        parsed_features = tf.io.parse_single_example(example_proto, keys_to_feature)
        if labeled:
            if return_labels:
                return parsed_features["TimeSeries"], parsed_features["Target"], parsed_features["id"]
            else:
                return parsed_features["TimeSeries"], parsed_features["Target"]
        else:
            return parsed_features["TimeSeries"]
    
    if not ordered:
        ignore_order = tf.data.Options()
        ignore_order.experimental_deterministic=False
        dataset = dataset.with_options(ignore_order)
    # parse the record into tensors.
    dataset = dataset.map(_parse_function, num_parallel_calls=AUTO)
#     dataset = dataset.cache()
    if cut:
        dataset = dataset.map(lambda x,y:tf.numpy_function(random_cut, inp=[x,y], Tout=[tf.float32, tf.int64]), num_parallel_calls=AUTO)
#     dataset = dataset.cache()

    # Repeat the input infinitely
    if repeat:
        dataset = dataset.repeat()

    # shuffle the dataset
    if shuffle:
        dataset = dataset.shuffle(buffer_size=10000, reshuffle_each_iteration=True)




#     # Generate batches
    dataset = dataset.batch(Params["batch_size"])
    dataset = dataset.prefetch(AUTO)
    return dataset

In [12]:
def get_train_val_files(folders):
    train_files = []
    val_files = []
    test_files = []
    all_train_files = []
    for folder in folders:
        if KAGGLE:
            TRAIN_FILES_PATH = KaggleDatasets().get_gcs_path(folder)
            TEST_FILES_PATH = KaggleDatasets().get_gcs_path(f"{folder}test")
            all_files_train = np.sort(tf.io.gfile.glob(f"{TRAIN_FILES_PATH}/train_*.tfrec"))
            all_files_test = np.sort(tf.io.gfile.glob(f"{TEST_FILES_PATH}/test_*.tfrec"))
        else:
            all_files_train = np.sort(glob(f"../input/{folder}/train_*.tfrec"))
            all_files_test = np.sort(glob(f"../input/{folder}/test_*.tfrec"))
        train_files.extend(all_files_train[:-2])
        val_files.extend(all_files_train[-2:])
        test_files.append(all_files_test)
        all_train_files.append(all_files_train)
    return train_files, val_files, test_files, all_train_files

train_files, val_files, test_files, all_train_files = get_train_val_files(tfrec_folders)


In [13]:
import time
def benchmark(dataset, num_epochs=2):
    start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # Performing a training step
            time.sleep(0.01)
    print("Execution time:", time.perf_counter() - start_time)
benchmark(train_files)

Execution time: 0.4364851999999999


In [14]:
train_ds = load_dataset(train_files, cut = True)
val_ds = load_dataset(val_files)

In [15]:
from tensorflow.keras import Sequential, layers
with strategy.scope():
    model = Sequential([
        layers.Conv1D(filters=32, kernel_size=16, padding="causal",input_shape=[4096,3]),
        layers.Conv1D(filters=32, kernel_size=16, padding="causal"),
        layers.MaxPool1D(pool_size=4),
        layers.Activation("relu"),
        
        layers.Conv1D(filters=64, kernel_size=8, padding="causal"),
        layers.Conv1D(filters=128, kernel_size=8, padding="causal"),
        layers.Conv1D(filters=128, kernel_size=8, padding="causal"),
        layers.MaxPool1D(pool_size=4),
        layers.Dropout(0.4),
        layers.Activation("relu"),
        
        layers.Conv1D(filters=128, kernel_size=8, padding="causal"),
        layers.Conv1D(filters=128, kernel_size=8, padding="causal"),
        layers.MaxPool1D(pool_size=4),
        layers.Activation("relu"),
        
        layers.Conv1D(filters=256, kernel_size=8, padding="causal"),
        layers.Conv1D(filters=256, kernel_size=8, padding="causal"),
        layers.MaxPool1D(pool_size=4),
        layers.Activation("relu"),
        
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(1),
        layers.Activation("sigmoid", dtype="float32")
    ])
    model.compile(
        tf.keras.optimizers.Adam(learning_rate = Params["lr"]),
        loss="binary_crossentropy",
        metrics=["AUC"]
    )
    model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 4096, 32)          1568      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 4096, 32)          16416     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1024, 32)          0         
_________________________________________________________________
activation (Activation)      (None, 1024, 32)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1024, 64)          16448     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1024, 128)         65664     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 1024, 128)         1

In [16]:
steps_per_epoch = 560000 // 16 * len(train_files) // Params["batch_size"]
validation_steps = 560000 // 16 * len(val_files) // Params["batch_size"]

In [17]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.2,
    patience=3, min_lr = 0.000001,
    verbose=1
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=5
)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    f"{MDL_PATH}/model_{Params['version']:03}.h5",
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weight_only=False,
    mode="auto",
    save_freq="epoch"
)

callbacks=[reduce_lr, early_stop, model_checkpoint]

In [ ]:
history = model.fit(train_ds, validation_data = val_ds, epochs = Params["epochs"], shuffle=True,
                    steps_per_epoch = steps_per_epoch, validation_steps=validation_steps,
                    verbose=1, callbacks=callbacks)

Epoch 1/100
1914/1914 [==============================] - 81s 39ms/step - loss: 0.6199 - auc: 0.6692 - val_loss: 0.5045 - val_auc: 0.8015
Epoch 2/100
1914/1914 [==============================] - 75s 39ms/step - loss: 0.4966 - auc: 0.8082 - val_loss: 0.4737 - val_auc: 0.8272
Epoch 3/100
1914/1914 [==============================] - 77s 40ms/step - loss: 0.4778 - auc: 0.8228 - val_loss: 0.4637 - val_auc: 0.8339
Epoch 4/100
1914/1914 [==============================] - 81s 42ms/step - loss: 0.4667 - auc: 0.8308 - val_loss: 0.4537 - val_auc: 0.8399
Epoch 5/100
1914/1914 [==============================] - 95s 50ms/step - loss: 0.4598 - auc: 0.8359 - val_loss: 0.4494 - val_auc: 0.8427TA: 4s - l - ETA: 1s - los - ETA: 0s - loss: 0.4598 - auc: 0.83
Epoch 6/100
 869/1914 [============>.................] - ETA: 51s - loss: 0.4571 - auc: 0.8376 ETA: 51s - loss: 0.

In [ ]:
historyFrame = pd.DataFrame(history.history)
historyFrame[["auc", "val_auc"]].plot()
historyFrame[["loss", "val_loss"]].plot()
historyFrame.to_csv(f"{MDL_PATH}/history_mdl{Params['version']:03}.csv", index=False)

In [ ]:
historyFrame

In [ ]:
best_epoch = historyFrame.val_auc.argmax()
best_loss = historyFrame.iloc[best_epoch].loss
best_auc = historyFrame.iloc[best_epoch].val_auc
print("best epoch:", best_epoch,
      "| best loss:", best_loss,
      "| best auc:", best_auc
     )

In [ ]:
result = Params.copy()
result["bavg_epoch"] = int(best_epoch)
result["bavg_loss"] = float(best_loss)
result["bavg_auc"] = float(best_auc)
with open(f"{MDL_PATH}/params.json", "w") as file:
    json.dump(result, file)

if not os.path.exists(f"{MDLS_PATH}/results.csv"):
    df_save = pd.DataFrame(result, index=[0])
    df_save.to_csv(f"{MDLS_PATH}/results.csv")
else:
    df_old = pd.read_csv(f"{MDLS_PATH}/results.csv", index_col=0)
    df_save = pd.DataFrame(result, index = [df_old.index.max() + 1])
    df_save = df_old.append(df_save, ignore_index=True)
    df_save.to_csv(f"{MDLS_PATH}/results.csv")   

In [ ]:
pd.read_csv(f"{MDLS_PATH}/results.csv",index_col=[0])

In [ ]:
del train_ds, val_ds
gc.collect()

In [ ]:
all_train_files

In [ ]:
prediction_scores = []
with strategy.scope():
    model = tf.keras.models.load_model(f"{MDL_PATH}/model_{Params['version']:03}.h5")
train_df = pd.read_csv("../input/g2net-gravitational-wave-detection/training_labels.csv")
for ds_ind in range(len(all_train_files)):
    train_set = load_dataset(all_train_files[ds_ind], shuffle=False, ordered=True, labeled=True, repeat=False, return_labels=False)
    prediction = model.predict(train_set)
    tf.keras.backend.clear_session()
    gc.collect()
    prediction_scores.append(roc_auc_score(train_df.target, prediction))
print(prediction_scores)
best_pred = np.array(prediction_scores).argmax()

In [ ]:
test_set = load_dataset(test_files[best_pred], shuffle=False, ordered=True, labeled=False, repeat=False, return_labels=False)
test_prediction = model.predict(test_set)
sub = pd.read_csv("../input/g2net-gravitational-wave-detection/sample_submission.csv")
sub.target = test_prediction.flatten()
sub.to_csv(f"{MDL_PATH}/submission.csv", index=False)